In [1]:
import pyodbc
import subprocess
import os
import shutil
import pandas as pd
import csv
from getpass import getuser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [18]:
driver = webdriver.Edge()
driver.get("https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php?menu=sci&menuN1=estfinafp&menuN2=NOID")


In [19]:
driver.find_element(By.XPATH, "/html/body/div[2]/div/div[3]/div/div/form/table/tbody/tr[3]/td[3]/input").click()

In [20]:
driver.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div/div[2]/div[2]/div[2]/div/div/table/tbody/tr[8]/td[2]/a").click()

In [28]:
input_year = 1
input_month = 2

tbody = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div/div[2]/div[2]/div/table[2]/tbody")
data = []
for td in tbody.find_elements(By.XPATH, "//tr[1]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)

print(data)

user_id = getuser()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div/div[2]/div/div[2]/div[2]/div/table[2]/tbody"}
  (Session info: MicrosoftEdge=111.0.1661.54)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF64AB7A5C2+12658]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF64AB12862+747634]
	(No symbol) [0x00007FF64A7ADD2E]
	(No symbol) [0x00007FF64A7EFB49]
	(No symbol) [0x00007FF64A7EFCD0]
	(No symbol) [0x00007FF64A82B4B7]
	(No symbol) [0x00007FF64A80F42F]
	(No symbol) [0x00007FF64A7E2791]
	(No symbol) [0x00007FF64A82874C]
	(No symbol) [0x00007FF64A80F1C3]
	(No symbol) [0x00007FF64A7E17D6]
	(No symbol) [0x00007FF64A7E09B5]
	(No symbol) [0x00007FF64A7E1F44]
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF64AA25530+147088]
	(No symbol) [0x00007FF64A8C91A6]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF64AA

In [ ]:
Ruta = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Archivos/Entrada/DtPb_SP_DetalleIngresoPorComisiones/{}.csv'.format(user_id,input_year+input_month)

with open (Ruta, 'a', newline = "") as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerows(data)

In [ ]:
project_id = 'v13513clinsfindataczprod00001'
dataset_id = 'bgqbi_finance001'
table_id = 'DtPb_SP_DetalleIngresoPorComisiones'
leading_rows = 0
field_delimiter = ";"
repl_carga = "false"
estruc_json = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Estructuras/{}.json'.format(user_id,table_id)
local_path_input = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Archivos/Entrada/'.format(user_id)
local_path_done = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Archivos/Procesado/'.format(user_id)
local_path_root = r'C:/temp'
local_file = ""
local_file_root = ""

shutil.copy(estruc_json,r'{}{}.json'.format(local_path_root,table_id))

str_acceso = "gcloud auth login"
process = subprocess.run(str_acceso, shell = True, capture_output = True)
returncode = (process.returncode)
mensaje = str(process.stdout)+str(process.stderr)
print(returncode,mensaje.replace('\\','/'))

local_path = r'{}{}/'.format(local_path_input,table_id)
for file in os.listdir(local_path):
    print(file)

    local_file_root = r'{}{}'.format(local_path_root,file)
    local_file = r'{}{}'.format(local_path,file)
    local_path_done = r'{}{}/{}'.format(local_path_done,table_id,file)

    shutil.copy(local_file,local_file_root)

    gs_path = r'gs://strczclinsfin001/Inputs/{}/{}'.format(table_id,file)

    carga_storage = 'gsutil cp {} {}'.format(local_file_root,gs_path)
    process = subprocess.run(carga_storage, shell = True, capture_output = True)
    returncode = (process.returncode)
    mensaje = str(process.stdout)+str(process.stderr)

    carga_bq = 'bq load --skip_leading_rows={} --field_delimiter="{}" --replace={} {}.{} {} {}'.format(leading_rows,field_delimiter,repl_carga,dataset_id,table_id,gs_path,r'{}{}.json'.format(local_path_root,table_id))
    process = subprocess.run(carga_bq, shell = True, capture_output = True)
    returncode = (process.returncode)
    mensaje = str(process.stdout)+str(process.stderr)
    print(returncode,mensaje.replace('\\','/'))

    os.remove(local_file_root)
    shutil.move(local_file,local_path_done)

os.remove(r'{}{}.json'.format(local_path_root,table_id))